In [1]:
import numpy as np
import numpy.ma as ma

from astropy.table import QTable

import dynesty
from dynesty import plotting as dyplot

import emcee
import corner

import sys
sys.path.insert(1,"/Users/kellydouglass/Documents/Research/Rotation_curves/RotationCurves/spirals/")
from DRP_rotation_curve import extract_data
from DRP_vel_map_functions import model_vel_map, vel_logL_BB, logL_BB, deproject_spaxel, calculate_chi2
#from DRP_vel_map_plottingFunctions import plot_rot_curve

import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
MANGA_FOLDER = '/Users/kellydouglass/Documents/Research/data/SDSS/dr16/manga/spectro/'
VEL_MAP_FOLDER = MANGA_FOLDER + 'analysis/v2_4_3/2.2.1/HYB10-GAU-MILESHC/'

# Sample galaxy properties

In [3]:
gal_ID = '7443-12705'
#gal_ID = '8486-12701'

manga_plate, manga_IFU = gal_ID.split('-')
gal_filename = VEL_MAP_FOLDER + manga_plate + '/' + manga_IFU + '/manga-' + gal_ID + '-MAPS-HYB10-GAU-MILESHC.fits.gz'

In [4]:
Ha_vel, Ha_vel_ivar, Ha_vel_mask, r_band, r_band_ivar = extract_data(gal_filename)

mr_band = ma.array(r_band, mask=Ha_vel_mask)
mHa_vel = ma.array(Ha_vel, mask=Ha_vel_mask)
mHa_vel_ivar = ma.array(Ha_vel_ivar, mask=Ha_vel_mask)

In [5]:
oneD_fit_file = '../spirals/DRPall-master_file_30.txt'

oneD_fit_parameters = QTable.read(oneD_fit_file, format='ascii.ecsv')

gal_oneD_fit_parameters_boolean = np.logical_and(oneD_fit_parameters['MaNGA_plate'] == int(manga_plate), 
                                                 oneD_fit_parameters['MaNGA_IFU'] == int(manga_IFU))

gal_oneD_fit_parameters_row = oneD_fit_parameters[gal_oneD_fit_parameters_boolean]

In [36]:
i_angle = np.arccos(gal_oneD_fit_parameters_row['ba'][0])
center = np.unravel_index(ma.argmax(mr_band), mr_band.shape)
v_sys = mHa_vel[center]
phi = gal_oneD_fit_parameters_row['phi'][0].value*np.pi/180
v_max = gal_oneD_fit_parameters_row['avg_v_max'][0].value
r_turn = gal_oneD_fit_parameters_row['avg_r_turn'][0].value
alpha = gal_oneD_fit_parameters_row['avg_alpha'][0]

# Find spaxel along semi-major axis
delta_x = int(center[1]*0.5)
delta_y = int(delta_x/np.tan(phi))
semi_major_axis_spaxel = tuple(np.subtract(center, (-delta_y, delta_x)))

# Check value along semi-major axis
if mHa_vel[semi_major_axis_spaxel] < 0:
    phi_guess = phi + np.pi
else:
    phi_guess = phi

pos_params = [v_sys, i_angle, center[0], center[1], phi_guess]
vel_params = [v_max, r_turn, alpha]
best_fit_params = pos_params + vel_params
'''
best_fit_values = {'v_sys':v_sys,
                   'ba':gal_oneD_fit_parameters_row['ba'][0],
                   'x0':center[0],
                   'y0':center[1],
                   'phi':phi_guess,
                   'r_turn':r_turn,
                   'v_max':v_max,
                   'alpha':alpha}
''';

print(best_fit_params)

[-11.739962, 1.0683978716829448, 37, 37, 0.6447036817356814, 272.1956986958439, 3.9922481117178874, 1.6526717602899932]


In [7]:
map_shape = mHa_vel.shape

In [8]:
H_0 = 100      # Hubble's Constant in units of h km/s/Mpc
c = 299792.458 # Speed of light in units of km/s

MANGA_FIBER_DIAMETER = 2*(1/60)*(1/60)*(np.pi/180) # angular fiber diameter (2") in radians
MANGA_SPAXEL_SIZE = 0.5*(1/60)*(1/60)*(np.pi/180)  # spaxel size (0.5") in radians


dist_to_galaxy_Mpc = c*gal_oneD_fit_parameters_row['redshift'][0]/H_0
dist_to_galaxy_kpc = dist_to_galaxy_Mpc*1000

pix_scale_factor = dist_to_galaxy_kpc*np.tan(MANGA_SPAXEL_SIZE)

# Functions for dynesty sampler

In [58]:
def uniform(a, b, u):
    """Given u in [0,1], return a uniform number in [a,b]."""
    return a + (b-a)*u


def jeffreys(a, b, u):
    """Given u in [0,1], return a Jeffreys random number in [a,b]."""
    return a**(1-u) * b**u


def prior_xforBB(u):
    """
    Priors for the parameters of the BB velocity curve model. 
    Required by the dynesty sampler.
    
    Parameters
    ----------
    u : ndarray
        Array of uniform random numbers between 0 and 1.
        
    Returns
    -------
    priors : ndarray
        Transformed random numbers giving prior ranges on model parameters.
    """
    v_sys    = uniform(-300, 300, u[0])
    i_angle  = uniform(0, np.pi, u[1])
    i_center = jeffreys(0, 74, u[2])
    j_center = jeffreys(0, 74, u[3])
    phi      = uniform(-np.pi, np.pi, u[4])
    v_max    = uniform(1., 1e5, u[5])
    r_turn   = uniform(0.1, 100., u[6])
    alpha    = uniform(np.nextafter(0, 1), 100., u[7])
    
    return v_sys, i_angle, i_center, j_center, phi, v_max, r_turn, alpha


def prior_xforBB_vel(u):
    """
    Priors for the parameters of the BB velocity curve model. 
    Required by the dynesty sampler.
    
    Parameters
    ----------
    u : ndarray
        Array of uniform random numbers between 0 and 1.
        
    Returns
    -------
    priors : ndarray
        Transformed random numbers giving prior ranges on model parameters.
    """
    v_max    = uniform(1., 1e5, u[0])
    r_turn   = uniform(0.1, 100., u[1])
    alpha    = uniform(np.nextafter(0, 1), 100., u[2])
    
    return v_max, r_turn, alpha

# Nested sampler

In [10]:
dsampler = dynesty.DynamicNestedSampler(vel_logL_BB, prior_xforBB_vel, ndim=3,
                                        logl_args=(pos_params, pix_scale_factor, mHa_vel, mHa_vel_ivar),
                                        nlive=2000,
                                        bound='multi',
                                        sample='auto')

dsampler.run_nested()
dres1 = dsampler.results

0it [00:00, ?it/s]Traceback (most recent call last):
  File "/Users/kellydouglass/opt/anaconda3/lib/python3.8/site-packages/dynesty/dynesty.py", line 939, in __call__
    return self.func(x, *self.args, **self.kwargs)
  File "/Users/kellydouglass/Documents/Research/Rotation_curves/RotationCurves/spirals/DRP_vel_map_functions.py", line 539, in vel_logL_BB
    lambda1 = model_vel_map(params, vel_map.shape, pix_scale, 'BB')
  File "/Users/kellydouglass/Documents/Research/Rotation_curves/RotationCurves/spirals/DRP_vel_map_functions.py", line 234, in model_vel_map
    r, theta[i,j] = deproject_spaxel((i,j), center, phi, i_angle)
KeyboardInterrupt
0it [00:23, ?it/s]

Exception while calling loglikelihood function:
  params: [7.60733950e+04 4.35941553e+01 8.44527961e+01]
  args: ([-11.739962, 1.0683978716829448, 37, 37, 0.6447036817356814], 0.4685401594333536, masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=1e+20,
  dtype=float32), masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
    

KeyboardInterrupt: 